In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
998469,2021-02-14,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
998470,2021-02-15,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
998471,2021-02-16,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
998472,2021-02-17,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
27167,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
27169,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
27171,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
27173,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
27175,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
619270,2021-02-14,Arkansas,Arkansas,5001,1926,37.00,5000,Arkansas,AR,Arkansas,State,3017804
619272,2021-02-15,Arkansas,Arkansas,5001,1929,37.00,5000,Arkansas,AR,Arkansas,State,3017804
619274,2021-02-16,Arkansas,Arkansas,5001,1930,38.00,5000,Arkansas,AR,Arkansas,State,3017804
619276,2021-02-17,Arkansas,Arkansas,5001,1933,38.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
998469,2021-02-14,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
998470,2021-02-15,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
998471,2021-02-16,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
998472,2021-02-17,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-02-18') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
394,2021-02-18,Snohomish,Washington,30230,523.00,53061,WA,County,822083,3677.24,63.62
786,2021-02-18,Cook,Illinois,467986,9775.00,17031,IL,County,5150233,9086.70,189.80
1177,2021-02-18,Orange,California,258182,3685.00,6059,CA,County,3175692,8129.94,116.04
1567,2021-02-18,Maricopa,Arizona,501792,8694.00,4013,AZ,County,4485414,11187.19,193.83
1957,2021-02-18,Los Angeles,California,1174897,19514.00,6037,CA,County,10039107,11703.20,194.38
...,...,...,...,...,...,...,...,...,...,...,...
998081,2021-02-18,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08
998210,2021-02-18,King,Texas,13,0.00,48269,TX,County,272,4779.41,0.00
998308,2021-02-18,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00
998402,2021-02-18,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
394,2021-02-18,Snohomish,Washington,30230,523.00,53061,WA,County,822083,3677.24,63.62,63.62,3677.24
786,2021-02-18,Cook,Illinois,467986,9775.00,17031,IL,County,5150233,9086.70,189.80,189.80,9086.70
1177,2021-02-18,Orange,California,258182,3685.00,6059,CA,County,3175692,8129.94,116.04,116.04,8129.94
1567,2021-02-18,Maricopa,Arizona,501792,8694.00,4013,AZ,County,4485414,11187.19,193.83,193.83,11187.19
1957,2021-02-18,Los Angeles,California,1174897,19514.00,6037,CA,County,10039107,11703.20,194.38,194.38,11703.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
998081,2021-02-18,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08,75.08,1651.65
998210,2021-02-18,King,Texas,13,0.00,48269,TX,County,272,4779.41,0.00,0.00,4779.41
998308,2021-02-18,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00,0.00,4238.26
998402,2021-02-18,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
704367,2021-02-18,Clarke,Alabama,3416,48.00,1025,AL,County,23622,14461.10,203.20,203.20,14461.10,1
647397,2021-02-18,Hale,Alabama,2036,62.00,1065,AL,County,14651,13896.66,423.18,423.18,13896.66,2
507081,2021-02-18,Lowndes,Alabama,1298,44.00,1085,AL,County,9726,13345.67,452.40,452.40,13345.67,3
449824,2021-02-18,Etowah,Alabama,13012,300.00,1055,AL,County,102268,12723.43,293.35,293.35,12723.43,4
377706,2021-02-18,Franklin,Alabama,3952,75.00,1059,AL,County,31362,12601.24,239.14,239.14,12601.24,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618297,2021-02-18,Sublette,Wyoming,671,7.00,56035,WY,County,9831,6825.35,71.20,71.20,6825.35,19
949965,2021-02-18,Platte,Wyoming,572,11.00,56031,WY,County,8393,6815.20,131.06,131.06,6815.20,20
853300,2021-02-18,Niobrara,Wyoming,147,2.00,56027,WY,County,2356,6239.39,84.89,84.89,6239.39,21
767354,2021-02-18,Lincoln,Wyoming,1194,11.00,56023,WY,County,19830,6021.18,55.47,55.47,6021.18,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
507081,2021-02-18,Lowndes,Alabama,1298,44.00,1085,AL,County,9726,13345.67,452.40,452.40,13345.67,3,1
647397,2021-02-18,Hale,Alabama,2036,62.00,1065,AL,County,14651,13896.66,423.18,423.18,13896.66,2,2
448500,2021-02-18,Clay,Alabama,1407,53.00,1027,AL,County,13235,10630.90,400.45,400.45,10630.90,22,3
546678,2021-02-18,Greene,Alabama,857,32.00,1063,AL,County,8111,10565.90,394.53,394.53,10565.90,24,4
216291,2021-02-18,Walker,Alabama,6403,239.00,1127,AL,County,63521,10080.13,376.25,376.25,10080.13,32,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545361,2021-02-18,Hot Springs,Wyoming,359,3.00,56017,WY,County,4413,8135.06,67.98,67.98,8135.06,16,19
748356,2021-02-18,Uinta,Wyoming,2021,12.00,56041,WY,County,20226,9992.09,59.33,59.33,9992.09,5,20
767354,2021-02-18,Lincoln,Wyoming,1194,11.00,56023,WY,County,19830,6021.18,55.47,55.47,6021.18,22,21
215280,2021-02-18,Teton,Wyoming,3307,9.00,56039,WY,County,23464,14093.93,38.36,38.36,14093.93,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,63.62,3677.24,14,21
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,63.62,3677.24,14,21
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,63.62,3677.24,14,21
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,63.62,3677.24,14,21
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,63.62,3677.24,14,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996110,2021-02-14,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
996111,2021-02-15,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
996112,2021-02-16,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
996113,2021-02-17,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
701685,2020-04-01,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,203.20,14461.10,44,1,2.00
701686,2020-04-02,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,203.20,14461.10,44,1,0.00
701687,2020-04-03,Clarke,Alabama,3,0.00,1025,AL,County,23622,12.70,0.00,203.20,14461.10,44,1,1.00
701688,2020-04-04,Clarke,Alabama,7,0.00,1025,AL,County,23622,29.63,0.00,203.20,14461.10,44,1,4.00
701689,2020-04-05,Clarke,Alabama,9,0.00,1025,AL,County,23622,38.10,0.00,203.20,14461.10,44,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
780078,2021-02-14,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00
780079,2021-02-15,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00
780080,2021-02-16,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00
780081,2021-02-17,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
505049,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,452.40,13345.67,1,3,1.00,0.00
505050,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,452.40,13345.67,1,3,0.00,0.00
505051,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,452.40,13345.67,1,3,0.00,0.00
505052,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,452.40,13345.67,1,3,0.00,0.00
505053,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,452.40,13345.67,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503394,2021-02-14,Albany,Wyoming,3854,11.00,56001,WY,County,38880,9912.55,28.29,28.29,9956.28,23,6,5.00,0.00
503395,2021-02-15,Albany,Wyoming,3854,11.00,56001,WY,County,38880,9912.55,28.29,28.29,9956.28,23,6,0.00,0.00
503396,2021-02-16,Albany,Wyoming,3867,11.00,56001,WY,County,38880,9945.99,28.29,28.29,9956.28,23,6,13.00,0.00
503397,2021-02-17,Albany,Wyoming,3871,11.00,56001,WY,County,38880,9956.28,28.29,28.29,9956.28,23,6,4.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-02-18') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
258047,2021-02-18,Imperial,California,26817,583.00,6025,CA,County,181215,14798.44,321.72,321.72,14798.44,1,2,20.00,0.00
1957,2021-02-18,Los Angeles,California,1174897,19514.00,6037,CA,County,10039107,11703.20,194.38,194.38,11703.20,2,5,2676.00,146.00
451100,2021-02-18,Inyo,California,1197,34.00,6027,CA,County,18039,6635.62,188.48,188.48,6635.62,3,30,6.00,0.00
57717,2021-02-18,Stanislaus,California,54728,916.00,6099,CA,County,550660,9938.62,166.35,166.35,9938.62,4,10,97.00,6.00
71514,2021-02-18,Tulare,California,47311,717.00,6107,CA,County,466195,10148.33,153.80,153.80,10148.33,5,9,867.00,20.00
26817,2021-02-18,Riverside,California,287200,3599.00,6065,CA,County,2470546,11624.96,145.68,145.68,11624.96,6,6,137.00,22.00
546620,2021-02-18,Kings,California,21720,213.00,6031,CA,County,152940,14201.65,139.27,139.27,14201.65,7,3,24.00,0.00
408953,2021-02-18,Merced,California,28385,384.00,6047,CA,County,277680,10222.20,138.29,138.29,10222.20,8,8,79.00,2.00
26119,2021-02-18,Fresno,California,93555,1349.00,6019,CA,County,999101,9363.92,135.02,135.02,9363.92,9,14,192.00,8.00
45955,2021-02-18,San Joaquin,California,65705,1021.00,6077,CA,County,762148,8621.03,133.96,133.96,8621.03,10,18,46.00,3.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
949516,2021-02-18,Lassen,California,5558,19.00,6035,CA,County,30573,18179.44,62.15,62.15,18179.44,36,1,12.00,0.00
258047,2021-02-18,Imperial,California,26817,583.00,6025,CA,County,181215,14798.44,321.72,321.72,14798.44,1,2,20.00,0.00
546620,2021-02-18,Kings,California,21720,213.00,6031,CA,County,152940,14201.65,139.27,139.27,14201.65,7,3,24.00,0.00
128771,2021-02-18,San Bernardino,California,283356,2492.00,6071,CA,County,2180085,12997.47,114.31,114.31,12997.47,13,4,620.00,35.00
1957,2021-02-18,Los Angeles,California,1174897,19514.00,6037,CA,County,10039107,11703.20,194.38,194.38,11703.20,2,5,2676.00,146.00
26817,2021-02-18,Riverside,California,287200,3599.00,6065,CA,County,2470546,11624.96,145.68,145.68,11624.96,6,6,137.00,22.00
161793,2021-02-18,Kern,California,101157,727.00,6029,CA,County,900202,11237.14,80.76,80.76,11237.14,26,7,308.00,37.00
408953,2021-02-18,Merced,California,28385,384.00,6047,CA,County,277680,10222.20,138.29,138.29,10222.20,8,8,79.00,2.00
71514,2021-02-18,Tulare,California,47311,717.00,6107,CA,County,466195,10148.33,153.80,153.80,10148.33,5,9,867.00,20.00
57717,2021-02-18,Stanislaus,California,54728,916.00,6099,CA,County,550660,9938.62,166.35,166.35,9938.62,4,10,97.00,6.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
258047,2021-02-18,Imperial,California,26817,583.00,6025,CA,County,181215,14798.44,321.72,321.72,14798.44,1,2,20.00,0.00
1957,2021-02-18,Los Angeles,California,1174897,19514.00,6037,CA,County,10039107,11703.20,194.38,194.38,11703.20,2,5,2676.00,146.00
451100,2021-02-18,Inyo,California,1197,34.00,6027,CA,County,18039,6635.62,188.48,188.48,6635.62,3,30,6.00,0.00
57717,2021-02-18,Stanislaus,California,54728,916.00,6099,CA,County,550660,9938.62,166.35,166.35,9938.62,4,10,97.00,6.00
71514,2021-02-18,Tulare,California,47311,717.00,6107,CA,County,466195,10148.33,153.80,153.80,10148.33,5,9,867.00,20.00
26817,2021-02-18,Riverside,California,287200,3599.00,6065,CA,County,2470546,11624.96,145.68,145.68,11624.96,6,6,137.00,22.00
546620,2021-02-18,Kings,California,21720,213.00,6031,CA,County,152940,14201.65,139.27,139.27,14201.65,7,3,24.00,0.00
408953,2021-02-18,Merced,California,28385,384.00,6047,CA,County,277680,10222.20,138.29,138.29,10222.20,8,8,79.00,2.00
26119,2021-02-18,Fresno,California,93555,1349.00,6019,CA,County,999101,9363.92,135.02,135.02,9363.92,9,14,192.00,8.00
45955,2021-02-18,San Joaquin,California,65705,1021.00,6077,CA,County,762148,8621.03,133.96,133.96,8621.03,10,18,46.00,3.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,321.72,14798.44,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,321.72,14798.44,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,321.72,14798.44,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,321.72,14798.44,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,321.72,14798.44,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3372,02/14/21,Lassen,5498,19.00,17983.19,62.15,62.15,18179.44,36,1,0.00,0.00
3373,02/15/21,Lassen,5498,19.00,17983.19,62.15,62.15,18179.44,36,1,0.00,0.00
3374,02/16/21,Lassen,5546,19.00,18140.19,62.15,62.15,18179.44,36,1,48.00,0.00
3375,02/17/21,Lassen,5546,19.00,18140.19,62.15,62.15,18179.44,36,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,321.72,14798.44,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,321.72,14798.44,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,321.72,14798.44,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,321.72,14798.44,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,321.72,14798.44,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3446,02/14/21,San Joaquin,64969,992.00,8524.46,130.16,133.96,8621.03,10,18,0.00,0.00
3447,02/15/21,San Joaquin,64969,992.00,8524.46,130.16,133.96,8621.03,10,18,0.00,0.00
3448,02/16/21,San Joaquin,65359,1018.00,8575.63,133.57,133.96,8621.03,10,18,390.00,26.00
3449,02/17/21,San Joaquin,65659,1018.00,8614.99,133.57,133.96,8621.03,10,18,300.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)